In [1]:
import requests
import bs4
import polars as pl
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

base = Path.home() / "Local_Workspace" / "Datasets" / "ARPA" / "FVG"

In [42]:
sample = base / "data" / "103400" / "2016.html"

soup = bs4.BeautifulSoup(sample.read_text())

# Clean up the text, removing the "\n\t" and 

In [43]:
def read_csv_table(path):
    soup = bs4.BeautifulSoup(path.read_text()[1:-1].replace(r"\n\t", "").replace(r"\n", "").replace('\\', ''))
    from urllib.parse import unquote
    csv = soup.find(id="salvaDati").attrs["href"][len("data:application/csv;charset=utf-8,"):]
    # Escapse the csv string from html codes
    return(unquote(csv))

In [58]:
from io import StringIO
pd.read_csv(StringIO(read_csv_table(sample)), sep=";", na_values=["-"], skipfooter=3, engine="python")

# List all the html files in the subdirectories of base / "data":
for station in (base / "data").glob("*/"):
    station_data = []
    for year in station.glob("*.html"):
        df = pd.read_csv(StringIO(read_csv_table(year)), sep=";", na_values=["-"], skipfooter=3, engine="python")
        df["anno"] = int(year.stem)
        station_data.append(df)
    station_data = pd.concat(station_data, axis=0)
    station_data["stazione"] = station.stem
    station_data.to_csv(station / "data.csv", index=False)

In [59]:
pd.read_csv(StringIO(read_csv_table(sample)), sep=";", na_values=["-"], engine="python")

,mese,giorno*,Pioggia mm,Temp. min gradi C,Temp. med gradi C,Temp. max gradi C,Umidita' min %,Umidita' med %,Umidita' max %,Vento med km/h,Vento max km/h,Dir. V. max gradi N,Radiaz. KJ/m2,Press. med hPa
0,1,1.0,0.0,-4.6,-0.6,7.0,43.0,81.0,97.0,2.0,8.0,360.0,6203.0,NaN
1,1,2.0,0.0,-5.9,-2.0,1.2,71.0,87.0,95.0,1.0,6.0,360.0,2014.0,NaN
2,1,3.0,5.6,-0.4,0.8,2.7,84.0,94.0,98.0,1.0,15.0,43.0,1510.0,NaN
3,1,4.0,1.2,-1.0,0.2,5.1,75.0,94.0,98.0,2.0,9.0,219.0,2624.0,NaN
4,1,5.0,0.2,-2.1,-0.6,2.1,84.0,95.0,98.0,2.0,10.0,360.0,1465.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,12,30.0,0.2,-4.7,-1.8,6.0,52.0,84.0,95.0,1.0,10.0,91.0,6368.0,NaN
365,12,31.0,0.0,-5.9,-2.4,6.7,40.0,80.0,94.0,1.0,6.0,58.0,6714.0,NaN
366,NOTA BENE: l'utilizzo dei dati e delle informa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,* dati dalle 00 alle 24 UTC (ora di Greenwich)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
html_files[0].stem

'2021'

In [6]:
pd.read_html(sample)[0]

,mese<\/th>,giorno*<\/th>,Pioggia mm<\/th>,Temp. min °C<\/th>,Temp. med °C<\/th>,Temp. max °C<\/th>,Umidità min %<\/th>,Umidità med %<\/th>,Umidità max %<\/th>,Vento med km\/h<\/th>,Vento max km\/h<\/th>,Dir. V. max °N<\/th>,Radiaz. KJ\/m2<\/th>,Press. med hPa<\/th><\/tr>\n\t\t\n\t\t<\/thead>\n\t \t
,<\/th>,totale<\/th>,minimo<\/th>,medio<\/th>,massimo<\/th>\n\t <\/tr>\n\t\t <\/thead>\n\t\t,Unnamed: 5_level_1,Unnamed: 6_level_1,Unnamed: 7_level_1,Unnamed: 8_level_1,Unnamed: 9_level_1,Unnamed: 10_level_1,Unnamed: 11_level_1,Unnamed: 12_level_1,Unnamed: 13_level_1
0,1<\/td>,1<\/td>,0.2<\/td>,-4.6<\/td>,-1.1<\/td>,5.3<\/td>,55<\/td>,87<\/td>,99<\/td>,0<\/td>,8<\/td>,196<\/td>,6093<\/td>,-<\/td><\/tr>
1,1<\/td>,2<\/td>,10.8<\/td>,-3.5<\/td>,-0.4<\/td>,1.7<\/td>,81<\/td>,95<\/td>,100<\/td>,0<\/td>,6<\/td>,53<\/td>,1402<\/td>,-<\/td><\/tr>
2,1<\/td>,3<\/td>,9.0<\/td>,0.7<\/td>,3.3<\/td>,7.0<\/td>,88<\/td>,98<\/td>,100<\/td>,0<\/td>,10<\/td>,90<\/td>,2885<\/td>,-<\/td><\/tr>
3,1<\/td>,4<\/td>,0.0<\/td>,-0.6<\/td>,1.1<\/td>,3.6<\/td>,-<\/td>,-<\/td>,-<\/td>,0<\/td>,9<\/td>,360<\/td>,1415<\/td>,-<\/td><\/tr>
4,1<\/td>,5<\/td>,1.4<\/td>,-1.4<\/td>,0.6<\/td>,3.4<\/td>,83<\/td>,96<\/td>,100<\/td>,1<\/td>,14<\/td>,66<\/td>,3803<\/td>,-<\/td><\/tr>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,precipitazione [mm]<\/td>,2470.8<\/td>,-<\/td>,-<\/td>,-<\/td><\/tr>\n\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,temperatura [\u00b0C]<\/td>,-<\/td>,-98.0<\/td>,10.2<\/td>,32.5<\/td><\/tr>\n\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,vento [km\/h]<\/td>,-<\/td>,-<\/td>,3<\/td>,54<\/td><\/tr>\n\t,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
